Sodelujoči

    Tomaž Štrus
    Anže Košir
    Jan Harej

O projektu: Iz platforme Steam, namenjenu predvsem za distrubicijo video iger, bomo zbrali podatke igrah več ljudi, ter analizirali stvari kot so npr. koliko iger, ki jih kupijo dejansko preigrajo, kolika časa bo nekdo zapravil za neko igro, kakšno igro bo kupil naslednjo, glede na svoje igre ipd.

Podatki: Uporabili bomo API, ki ga ponuja Steam: https://developer.valvesoftware.com/wiki/Steam_Web_API#JSON Tu lahko dobimo podatko o čisto vsemu kar potrebujemo npr. o uporabniku, njegove igre, prijatelji, koliko ur je igral neko igro, ... Dobili bomo podatke za npr. 100.000+ uporabnikov (prijatelji naših prijateljev, in njihov prijateljev itd.)

## Vprašanja za raziskavo
1. Povprečno dokončanje igre
2. Število in vrsta Bannov v povezavi z javnimi/privatnimi profili
3. Najmanjkrat odklenjeni dosežki za igrane igre
4. Število prijateljev glede na "starost" računa

In [3]:
#install libraries on windows
"""
!python -m pip install numpy
!python -m pip install matplotlib
!python -m pip install requests
!python -m pip install aiohttp
!python -m pip install asyncio
!python -m pip install Cchardet
"""

You are using pip version 10.0.0, however version 10.0.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [2]:
from scripts.imports import *
from scripts.data_variables import *
from scripts.gameStats_functions import *
from scripts.get_read_write import *

In [3]:
data = read_all_from_file()
player_ids = data[0] 
player_summaries = data[1]
player_friends = data[2]
player_games = data[3]
player_achievements = data[4]
player_bans = data[5]
global_game_stats = data[6]
game_names= data[7]
#player_ids = read_player_ids()
print("Current database size: {} users".format(len(player_ids)))
#player_summaries = read_player_summaries()
#player_friends = read_player_friends()
#player_games = read_player_games()
#player_bans = read_player_bans()

INFO: reading ids
INFO: reading ids finished
INFO: reading summaries
INFO: reading summaries finished
INFO: reading friends
INFO: reading friends finished
INFO: reading games
INFO: reading games finished
INFO: reading achievements
INFO: reading achievements finished
INFO: reading bans
INFO: reading bans finished
INFO: reading global game stats
INFO: reading global game stats finished
INFO: reading game names
INFO: reading game names finished
Current database size: 100000 users


# Odgovori na vprašanja

### 1. Povprečno dokončanje igre

### 2. Število in vrsta Bannov v povezavi z javnimi/privatnimi profili

### 3. Najmanjkrat odklenjeni dosežki za igrane igre

### 4. Število prijateljev glede na "starost" računa

In [4]:
player_summaries

{'76561198139420997': {'cvs': 3,
  'name': 'oklahomaops',
  'pcid': '103582791461071503',
  'ps': 1,
  'tc': 1402015294},
 '76561198172047405': {'cvs': 1, 'name': 'SexyCano', 'ps': 1},
 '76561198154875421': {'cvs': 3,
  'name': 'GermanDoggo',
  'pcid': '103582791460691368',
  'ps': 1,
  'tc': 1410956197},
 '76561198099196103': {'cntry': 'FR',
  'cvs': 3,
  'name': 'DKT TeenL!Fe.',
  'pcid': '103582791431256639',
  'ps': 1,
  'tc': 1374409172},
 '76561198142981660': {'cntry': 'CA',
  'cvs': 3,
  'name': 'Nibba',
  'pcid': '103582791456984501',
  'ps': 1,
  'tc': 1404161315},
 '76561198198840622': {'cntry': 'RU',
  'cvs': 3,
  'name': 'Чувак с вак',
  'pcid': '103582791459912012',
  'ps': 1,
  'tc': 1432894366},
 '76561198088912689': {'cntry': 'TH',
  'cvs': 3,
  'name': 'ExilROF',
  'pcid': '103582791433435701',
  'ps': 1,
  'rn': 'BlackHeart',
  'tc': 1365924636},
 '76561198237781262': {'cntry': 'LV',
  'cvs': 3,
  'name': 'HusKii',
  'pcid': '103582791454841880',
  'ps': 1,
  'rn': 'M

In [5]:
graph = collections.defaultdict(int)
today = datetime.datetime.now()
treshold = 1
treshold *= round(365 / 7)

for player_id in player_ids:
    try:
        time_created = int(player_summaries[player_id]["tc"])
    except KeyError:
        continue
    
    print(player_friends[player_id])
    for tmp_friend in player_friends[player_id]:
        fr_si = int(tmp_friend["fs"])
        time_diff = fr_si - time_created
        weeks = round(time_diff / (3600 * 24 *7))
        
        # If the player made this friend in the first year (since the account was created)
        if 0 < weeks <= 52:
            graph[weeks] += 1

graph_x = []
graph_y = []

for week in sorted(graph):
    graph_x += [week]
    graph_y += [graph[week]]

fig = plt.figure()
axes = fig.add_axes([0.1, 0.1, 2, 1])
axes.bar(graph_x, graph_y)
axes.set_ylabel('št. prijateljev')
axes.set_xlabel('starost računa (tedni)')

{'fs': 1518700454, 'id': '76561198813132480'}


TypeError: string indices must be integers

## 5. Odvisnost med težavnostjo igre in njenim uspehom